# Auto-Enable Flow Logs

This will kick off templates that will auto-enable VPC flow logs across the organization.

In [16]:
from jupyterirtools import sso
import os
import time
import importlib
import json

sso.login("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=WLTG-FTFP to activate the session


<IPython.core.display.Javascript object>

### Prerequisite: Create a flow logs bucket
A destination bucket needs to exist before the flow logs can be added. Typically this is a bucket in the logging account.

Options:
* Option 1: Choose Existing Bucket
  * Find an existing bucket with a bucket policy that allows cloudwatch logs to write to the bucket.
  * Save the bucket name to the SSM Parameter 'Jupyter-Flowlogs-Bucket' in the management account.
* Option 2: Manually create the bucket
  * Create a bucket with the appropriate permissions
  * Save the bucket name to the SSM Parameter 'Jupyter-Flowlogs-Bucket' in the management account.
* Option 3: Run the create logging bucket notebook
  * Run the [create logging notebook](create_flowlogs_bucket.ipynb):



In [17]:
# Validate the SSM Parameters exists for 'Jupyter-Flowlogs-Bucket' in the management account.
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])
ssm_client = session.client('ssm')

try:
    flowlogs_bucket_response = ssm_client.get_parameter(Name='Jupyter-Flowlogs-Bucket')
    flowlogs_bucket = flowlogs_bucket_response['Parameter']['Value']
    print(f"flowlogs_bucket={flowlogs_bucket} configured. Continue to the next step.")
except ssm_client.exceptions.ParameterNotFound:
    print("Parameter Jupyter-Flowlogs-Bucket not found. Confirm configuration of the flow logs bucket.")

    


flowlogs_bucket=flowlogs-bucket-cfncloudtraillogsbucket-1be4gsnc376f0 configured. Continue to the next step.


In [18]:
from jupyterirtools import cloudformation
import importlib
import boto3
importlib.reload(cloudformation)

org_client = boto3.client('organizations')
desc_client = org_client.describe_organization()
org_unit_id = org_client.list_roots()['Roots'][0]['Id']

org_id = desc_client['Organization']['Id']

cfn_bucket_stack = cloudformation.deploy(stack_name="cfn-bucket-stack", 
                      role="Jupyter-IR-AdministratorAccess", 
                      account=os.environ['MANAGEMENT_ACCOUNT'], 
                      template="../cfn-templates/cfn-bucket.yaml",
                      parameters={
                          "OrgId": org_id
                      })

bucket_name = cfn_bucket_stack['Outputs']['CfnBucket']

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


aws cloudformation deploy --stack-name=cfn-bucket-stack --template-file ../cfn-templates/cfn-bucket.yaml --capabilities CAPABILITY_IAM --profile Jupyter-IR-AdministratorAccess-383086473915 --parameter-overrides  OrgId=o-krcf45heml

Waiting for changeset to be created..

No changes to deploy. Stack cfn-bucket-stack is up to date


In [20]:
import random, string

cfn_profile = f"Jupyter-IR-AdministratorAccess-{os.environ['MANAGEMENT_ACCOUNT']}"
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])

random_str=''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
!aws s3 sync ../cfn-templates/ s3://$bucket_name/$random_str/ --profile $cfn_profile > /dev/null

template_url = f'https://s3.us-east-1.amazonaws.com/{bucket_name}/{random_str}'

In [ ]:
flow_log_remediation_stack = cloudformation.deploy(stack_name="jupyter-ir-config", 
                      role="Jupyter-IR-AdministratorAccess", 
                      account=os.environ['MANAGEMENT_ACCOUNT'], 
                      template="../cfn-templates/organization-ir-config.yaml",
                      parameters={
                          "FlowLogsBucket": bucket_name,
                          "AutomateRemediationTemplateUrl": template_url,
                          "OrgUnitId": org_unit_id,
                          "OutputFormat": "parquet",
                      })

flow_log_remediation_stack

aws cloudformation deploy --stack-name=jupyter-ir-config --template-file ../cfn-templates/organization-ir-config.yaml --capabilities CAPABILITY_IAM --profile Jupyter-IR-AdministratorAccess-383086473915 --parameter-overrides  FlowLogsBucket=cfn-bucket-stack-cfnbucket-1l1tf6muc97gf AutomateRemediationTemplateUrl=https://s3.us-east-1.amazonaws.com/cfn-bucket-stack-cfnbucket-1l1tf6muc97gf/TUGHKV1C OrgUnitId=r-nvfg OutputFormat=parquet

Waiting for changeset to be created..
Waiting for stack create/update to complete


!aws cloudformation deploy --stack-name jupyter-ir-config --template-file cfn-templates/organization-ir-config.yaml --parameter-overrides FlowLogsBucket=flow_logs_bucket ConfigTopic=$config_topic ConfigBucket=$config_bucket OrgUnitId=$org_unit_id OutputFormat=$flowlog_output_format  AutomateRemediationTemplateUrl=$template_url RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_NAMED_IAM
